In [1]:
import numpy as np 
import pandas as pd

# Data Preparation

In [2]:
bd=pd.read_csv("/kaggle/input/final-training/training_data.layer_one_bot.csv")
bd['is_bot'] ="True"
hd=pd.read_csv("/kaggle/input/final-training/training_data.layer_one_human.csv")
hd["is_bot"]="False"

In [3]:
hd["is_bot"]

0      False
1      False
2      False
3      False
4      False
       ...  
217    False
218    False
219    False
220    False
221    False
Name: is_bot, Length: 222, dtype: object

In [4]:
data= pd.concat([hd, bd], ignore_index=True)

In [5]:
data.columns

Index(['_id', 'is_abuser', 'time_taken', 'typing_speed', 'mouse_distance',
       'is_proxy', 'state', 'country', 'user_agent', 'is_bot'],
      dtype='object')

In [6]:
data["is_proxy"]="false"

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _id             738 non-null    object 
 1   is_abuser       738 non-null    bool   
 2   time_taken      738 non-null    float64
 3   typing_speed    663 non-null    float64
 4   mouse_distance  738 non-null    float64
 5   is_proxy        738 non-null    object 
 6   state           737 non-null    object 
 7   country         737 non-null    object 
 8   user_agent      738 non-null    object 
 9   is_bot          738 non-null    object 
dtypes: bool(1), float64(3), object(6)
memory usage: 52.7+ KB


In [8]:
data["is_bot"]

0      False
1      False
2      False
3      False
4      False
       ...  
733     True
734     True
735     True
736     True
737     True
Name: is_bot, Length: 738, dtype: object

In [9]:
!pip install pyyaml ua-parser user-agents

In [10]:
from user_agents import parse

In [11]:
data["user_agent"]

0      Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1      Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2      Mozilla/5.0 (Linux; Android 10; K) AppleWebKit...
3      Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4      Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
                             ...                        
733    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
734    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
735    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
736    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
737    Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
Name: user_agent, Length: 738, dtype: object

In [12]:
def parse_user_agent(ua_string):
    ua = parse(ua_string)
    return {
        'browser_family': ua.browser.family,
        'browser_version': ua.browser.version_string,
        'os_family': ua.os.family,
        'os_version': ua.os.version_string,
        'device_family': ua.device.family,
        'is_mobile': ua.is_mobile,
        'is_tablet': ua.is_tablet,
        'is_pc': ua.is_pc,
        
    }

user_agent_features = data['user_agent'].apply(parse_user_agent).apply(pd.Series)
data = pd.concat([data, user_agent_features], axis=1)
data = data.drop(columns=['user_agent'])

In [13]:
data

,_id,is_abuser,time_taken,typing_speed,mouse_distance,is_proxy,state,country,is_bot,browser_family,browser_version,os_family,os_version,device_family,is_mobile,is_tablet,is_pc
0,66f7b99e22c5ebac6db3e9d5,False,3.959,6.756757,487.460044,false,Delhi,India,False,Chrome,129.0.0,Windows,10,Other,False,False,True
1,66f7b99e22c5ebac6db3e9d8,False,6.936,3.031974,584.761313,false,Delhi,India,False,Chrome,129.0.0,Windows,10,Other,False,False,True
2,66f7bce222c5ebac6db3e9da,True,10.297,2.388389,88.734256,false,Delhi,India,False,Chrome Mobile,128.0.0,Android,10,K,True,False,False
3,66f7c03022c5ebac6db3e9dd,False,10.789,2.835271,745.317058,false,Delhi,India,False,Chrome,128.0.0,Windows,10,Other,False,False,True
4,66f7c04522c5ebac6db3e9e0,False,4.743,3.458878,1031.050701,false,Delhi,India,False,Chrome,128.0.0,Windows,10,Other,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,66f79bb86e961372ec4a6ef2,False,8.795,1.776830,52.000000,false,Haryana,India,True,Chrome,128.0.0,Linux,,Other,False,False,True
734,66f79bcd6e961372ec4a6ef7,False,8.448,2.335994,52.000000,false,Haryana,India,True,Chrome,128.0.0,Linux,,Other,False,False,True
735,66f79be36e961372ec4a6efa,False,7.538,2.074689,52.000000,false,Haryana,India,True,Chrome,128.0.0,Linux,,Other,False,False,True
736,66f79bf86e961372ec4a6efe,False,8.055,2.520797,52.000000,false,Haryana,India,True,Chrome,128.0.0,Linux,,Other,False,False,True


In [14]:
data["typing_speed"].fillna(0)

0      6.756757
1      3.031974
2      2.388389
3      2.835271
4      3.458878
         ...   
733    1.776830
734    2.335994
735    2.074689
736    2.520797
737    1.932876
Name: typing_speed, Length: 738, dtype: float64

In [15]:
data

,_id,is_abuser,time_taken,typing_speed,mouse_distance,is_proxy,state,country,is_bot,browser_family,browser_version,os_family,os_version,device_family,is_mobile,is_tablet,is_pc
0,66f7b99e22c5ebac6db3e9d5,False,3.959,6.756757,487.460044,false,Delhi,India,False,Chrome,129.0.0,Windows,10,Other,False,False,True
1,66f7b99e22c5ebac6db3e9d8,False,6.936,3.031974,584.761313,false,Delhi,India,False,Chrome,129.0.0,Windows,10,Other,False,False,True
2,66f7bce222c5ebac6db3e9da,True,10.297,2.388389,88.734256,false,Delhi,India,False,Chrome Mobile,128.0.0,Android,10,K,True,False,False
3,66f7c03022c5ebac6db3e9dd,False,10.789,2.835271,745.317058,false,Delhi,India,False,Chrome,128.0.0,Windows,10,Other,False,False,True
4,66f7c04522c5ebac6db3e9e0,False,4.743,3.458878,1031.050701,false,Delhi,India,False,Chrome,128.0.0,Windows,10,Other,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,66f79bb86e961372ec4a6ef2,False,8.795,1.776830,52.000000,false,Haryana,India,True,Chrome,128.0.0,Linux,,Other,False,False,True
734,66f79bcd6e961372ec4a6ef7,False,8.448,2.335994,52.000000,false,Haryana,India,True,Chrome,128.0.0,Linux,,Other,False,False,True
735,66f79be36e961372ec4a6efa,False,7.538,2.074689,52.000000,false,Haryana,India,True,Chrome,128.0.0,Linux,,Other,False,False,True
736,66f79bf86e961372ec4a6efe,False,8.055,2.520797,52.000000,false,Haryana,India,True,Chrome,128.0.0,Linux,,Other,False,False,True


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers
import joblib
label_enc = LabelEncoder()
data['country'] = label_enc.fit_transform(data['country'])
joblib.dump(label_enc, 'country_label_encoder.joblib')
label_enc = LabelEncoder()
data['state'] = label_enc.fit_transform(data['state'])
joblib.dump(label_enc, 'city_label_encoder.joblib')
label_enc = LabelEncoder()
data['is_proxy'] = label_enc.fit_transform(data['is_proxy'])
joblib.dump(label_enc, 'proxy_label_encoder.joblib')
label_enc = LabelEncoder()
data['is_abuser'] = label_enc.fit_transform(data['is_abuser'])
joblib.dump(label_enc, 'abuser_label_encoder.joblib')
label_enc = LabelEncoder()

for column in ['browser_family', 'os_family', 'device_family','browser_version','os_version']:
    label_enc = LabelEncoder()
    data[column] = label_enc.fit_transform(data[column])
    joblib.dump(label_enc, f'{column}_label_encoder.joblib')
   

data['is_mobile'] = data['is_mobile'].astype(int)
data['is_tablet'] = data['is_tablet'].astype(int)
data['is_pc'] = data['is_pc'].astype(int)


In [17]:
data

,_id,is_abuser,time_taken,typing_speed,mouse_distance,is_proxy,state,country,is_bot,browser_family,browser_version,os_family,os_version,device_family,is_mobile,is_tablet,is_pc
0,66f7b99e22c5ebac6db3e9d5,0,3.959,6.756757,487.460044,0,0,0,False,0,3,3,1,2,0,0,1
1,66f7b99e22c5ebac6db3e9d8,0,6.936,3.031974,584.761313,0,0,0,False,0,3,3,1,2,0,0,1
2,66f7bce222c5ebac6db3e9da,1,10.297,2.388389,88.734256,0,0,0,False,1,2,0,1,0,1,0,0
3,66f7c03022c5ebac6db3e9dd,0,10.789,2.835271,745.317058,0,0,0,False,0,2,3,1,2,0,0,1
4,66f7c04522c5ebac6db3e9e0,0,4.743,3.458878,1031.050701,0,0,0,False,0,2,3,1,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,66f79bb86e961372ec4a6ef2,0,8.795,1.776830,52.000000,0,1,0,True,0,2,1,0,2,0,0,1
734,66f79bcd6e961372ec4a6ef7,0,8.448,2.335994,52.000000,0,1,0,True,0,2,1,0,2,0,0,1
735,66f79be36e961372ec4a6efa,0,7.538,2.074689,52.000000,0,1,0,True,0,2,1,0,2,0,0,1
736,66f79bf86e961372ec4a6efe,0,8.055,2.520797,52.000000,0,1,0,True,0,2,1,0,2,0,0,1


In [18]:
data=data.drop(["_id","is_proxy"],axis=1)

In [19]:
data['is_bot'] = data['is_bot'].map({"True":1,"False":0})

X = data.drop('is_bot', axis=1)
y = data['is_bot']

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   is_abuser        738 non-null    int64  
 1   time_taken       738 non-null    float64
 2   typing_speed     663 non-null    float64
 3   mouse_distance   738 non-null    float64
 4   state            738 non-null    int64  
 5   country          738 non-null    int64  
 6   browser_family   738 non-null    int64  
 7   browser_version  738 non-null    int64  
 8   os_family        738 non-null    int64  
 9   os_version       738 non-null    int64  
 10  device_family    738 non-null    int64  
 11  is_mobile        738 non-null    int64  
 12  is_tablet        738 non-null    int64  
 13  is_pc            738 non-null    int64  
dtypes: float64(3), int64(11)
memory usage: 80.8 KB


# Splitting

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

# XGB training

In [23]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_model.save_model('demo_weights_xgboost.json')
xgb_preds = xgb_model.predict(X_test)
print(xgb_preds)
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_preds))
print(classification_report(y_test, xgb_preds))

[1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1
 1 0 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 0 1 0 0 0 1 1 1 0 1 0 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 1 0
 1 0 0 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 0 1
 0 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1]
XGBoost Accuracy: 0.9891891891891892
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        56
           1       1.00      0.98      0.99       129

    accuracy                           0.99       185
   macro avg       0.98      0.99      0.99       185
weighted avg       0.99      0.99      0.99       185



# CatBoost Training

In [24]:
catboost_model = CatBoostClassifier(verbose=0)
catboost_model.fit(X_train, y_train)
catboost_model.save_model('demo_weights_catboost.json')
catboost_preds = catboost_model.predict(X_test)
print("CatBoost Accuracy:", accuracy_score(y_test, catboost_preds))
print(classification_report(y_test, catboost_preds))

CatBoost Accuracy: 0.9837837837837838
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        56
           1       1.00      0.98      0.99       129

    accuracy                           0.98       185
   macro avg       0.97      0.99      0.98       185
weighted avg       0.98      0.98      0.98       185



# LGBM Training

In [25]:
import lightgbm as lgb
lgbm_model = lgb.LGBMClassifier()
lgbm_model.fit(X_train, y_train)
lgbm_model.booster_.save_model('demo_weights_lgbm.txt')
joblib.dump(lgbm_model, 'lgbm_model.joblib')
lgbm_preds = lgbm_model.predict(X_test)
print(lgbm_preds)
print("LightGBM Accuracy:", accuracy_score(y_test, lgbm_preds))
print(classification_report(y_test, lgbm_preds))

[LightGBM] [Info] Number of positive: 387, number of negative: 166
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002921 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 568
[LightGBM] [Info] Number of data points in the train set: 553, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.699819 -> initscore=0.846437
[LightGBM] [Info] Start training from score 0.846437
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

# Mixture of Experts(MoE) Approach

In [26]:
xgb_preds_proba_train = xgb_model.predict_proba(X_train)[:, 1]  
catboost_preds_proba_train = catboost_model.predict_proba(X_train)[:, 1]
lgbm_model_preds_proba_train = lgbm_model.predict_proba(X_train)[:, 1] 

xgb_preds_proba_test = xgb_model.predict_proba(X_test)[:, 1]
catboost_preds_proba_test = catboost_model.predict_proba(X_test)[:, 1]
lgbm_model_preds_proba_test = lgbm_model.predict_proba(X_test)[:, 1] 

train_expert_predictions = np.column_stack((xgb_preds_proba_train, catboost_preds_proba_train,lgbm_model_preds_proba_train))
test_expert_predictions = np.column_stack((xgb_preds_proba_test, catboost_preds_proba_test,lgbm_model_preds_proba_test))


In [27]:
def build_gate_model():
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(3,)),
        layers.Dense(32),
        layers.Dense(16, activation='relu'),
        layers.Dense(8),
        layers.Dense(1, activation='sigmoid')  
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

gate_model = build_gate_model()

gate_model.fit(train_expert_predictions, y_train, epochs=20, batch_size=32, verbose=0)
gate_model.save_weights("demo_weights_gate.weights.h5")
gate_preds_proba_test = gate_model.predict(test_expert_predictions)
gate_preds_test = (gate_preds_proba_test > 0.5).astype(int) 

print("Neural Network Gate Model (MoE) Accuracy:", accuracy_score(y_test, gate_preds_test))
print(classification_report(y_test, gate_preds_test))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Neural Network Gate Model (MoE) Accuracy: 0.9945945945945946
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        56
           1       1.00      0.99      1.00       129

    accuracy                           0.99       185
   macro avg       0.99      1.00      0.99       185
weighted avg       0.99      0.99      0.99       185

